In [1]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm
# !pip install catboost
import catboost

# Data 준비

negative 생성

In [2]:
base_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')
ease_df = pd.read_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/Recbole/EASE_best_1.csv') # 0.1594
recvae_df = pd.read_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/RecVAE_cms.csv') # 0.0963
neu_df = pd.read_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/NeuMF_rslt.csv') # 0.1048
ffm_df = pd.read_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/FFM_rslt.csv') # 0.0999
sli_df = pd.read_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/Slim.csv') # 0.1279

In [3]:
base_df['rating'] = 1

# 3. Negative instance 생성 : 각 유저별 true:false = 5:5, 20분정도 소요
print("Create Nagetive instances")
# num_negative = 50
user_group_dfs = list(base_df.groupby('user')['item'])

first_row = True
items = set(base_df.loc[:, 'item'])

for u, u_items in tqdm(user_group_dfs):
    u_items = set(u_items)                 #해당 유저가 본 아이템
    i_user_neg_item = np.random.choice(list(items - u_items), len(u_items), replace=False)
    
    i_user_neg_df = pd.DataFrame({'user': [u]*len(u_items), 'item': i_user_neg_item, 'rating': [0]*len(u_items)})
    if first_row == True:
        user_neg_dfs = i_user_neg_df
        first_row = False
    else:
        user_neg_dfs = pd.concat([user_neg_dfs, i_user_neg_df], axis = 0, sort=False)

result = pd.concat([base_df, user_neg_dfs], axis = 0, sort=False)
result.to_csv("user_direct_neg_sampling.csv", index=False)
result.isna().sum()

Create Nagetive instances


 60%|█████▉    | 18772/31360 [06:55<04:38, 45.18it/s]


KeyboardInterrupt: 

In [8]:
base_w_neg_df = pd.read_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/model/user_direct_neg_sampling.csv')
base_w_neg_df = pd.concat([base_w_neg_df, ease_df, ease_df]).drop_duplicates(subset=['user', 'item'], keep=False)
base_w_neg_df = pd.concat([base_w_neg_df, recvae_df, recvae_df]).drop_duplicates(subset=['user', 'item'], keep=False)
base_w_neg_df = pd.concat([base_w_neg_df, neu_df, neu_df]).drop_duplicates(subset=['user', 'item'], keep=False)
base_w_neg_df = pd.concat([base_w_neg_df, ffm_df, ffm_df]).drop_duplicates(subset=['user', 'item'], keep=False)
base_w_neg_df = pd.concat([base_w_neg_df, sli_df, sli_df]).drop_duplicates(subset=['user', 'item'], keep=False)
base_w_neg_df.drop('time', axis=1, inplace=True)

In [9]:
base_w_neg_df

,user,item,rating
0,11,4643,1.0
1,11,170,1.0
2,11,531,1.0
3,11,616,1.0
4,11,2140,1.0
...,...,...,...
10308937,138493,3177,0.0
10308938,138493,4029,0.0
10308939,138493,5873,0.0
10308940,138493,52375,0.0


FE을 통행 생성한 column들을 추가한다

In [10]:
full_year = pd.read_csv("/opt/ml/input/code/data/year_missingno.csv")
writ = pd.read_csv("/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/fullwri.csv")
director = pd.read_csv("/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/director_str.csv")
genre = pd.read_csv("/opt/ml/input/code/data/genre_bert_fe.csv")
pop_dir = pd.read_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/ref_data/item_popular_dir_max_sum_cnt.csv')
pop_wri = pd.read_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/ref_data/item_popular_wri_max_sum_cnt.csv')
user_seen = pd.read_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/ref_data/user_seen_item_statistic.csv')

def dir_one(aaa):
    return aaa.split(' ')[0]
director['director_str'] = director['director_str'].apply(dir_one)

merge_df = pd.merge(base_w_neg_df, full_year, on='item', how='left')
merge_df = pd.merge(merge_df, writ, on='item', how='left')
merge_df = pd.merge(merge_df, director, on='item', how='left')
merge_df = pd.merge(merge_df, genre, on='item', how='left')
merge_df = pd.merge(merge_df, pop_dir, on='item', how='left')
merge_df = pd.merge(merge_df, pop_wri, on='item', how='left')
merge_df = pd.merge(merge_df, user_seen, on='user', how='left')
merge_df.drop(['Unnamed: 0', 'writer_inter_x',	'writer_inter_y','writer_info'], axis=1, inplace=True)
merge_df.isna().sum()

user                    0
item                    0
rating                  0
year                    0
writer                  0
director_str            0
genre_embedding         0
dir_max_cnt             0
dir_sum_cnt             0
wri_max_cnt             0
wri_sum_cnt             0
seen_item_inter_mean    0
seen_item_inter_std     0
dtype: int64

In [11]:
merge_df

,user,item,rating,year,writer,director_str,genre_embedding,dir_max_cnt,dir_sum_cnt,wri_max_cnt,wri_sum_cnt,seen_item_inter_mean,seen_item_inter_std
0,11,4643,1.0,2001.0,nm0099541,nm0000318,-0.010404,7743,51470,8936,21339,3719.351064,4316.497874
1,11,170,1.0,1995.0,nm0000000,nm0812200,-0.010290,2092,4086,1107,1107,3719.351064,4316.497874
2,11,531,1.0,1993.0,nm0003031,nm0002140,-0.010649,966,1291,7743,19336,3719.351064,4316.497874
3,11,616,1.0,1970.0,nm0027011,nm0718627,-0.010579,2052,10798,2052,11529,3719.351064,4316.497874
4,11,2140,1.0,1982.0,nm0001345,nm0001345,-0.010351,2175,9200,2175,4005,3719.351064,4316.497874
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10285234,138493,3177,0.0,2000.0,nm0001084,nm0139867,-0.010054,159,455,747,906,3701.936306,4167.737972
10285235,138493,4029,0.0,2000.0,nm0000519,nm0000519,-0.010281,506,2209,4748,15704,3701.936306,4167.737972
10285236,138493,5873,0.0,2002.0,nm0866063,nm0001355,-0.010508,338,864,218,313,3701.936306,4167.737972
10285237,138493,52375,0.0,2007.0,nm0000000,nm0002120,-0.010337,3571,10867,163,163,3701.936306,4167.737972


In [12]:
user2idx = {v:k for k,v in enumerate(sorted(set(merge_df.user)))}
item2idx = {v:k for k,v in enumerate(sorted(set(merge_df.item)))}
writer2idx = {v:k for k,v in enumerate(sorted(set(merge_df.writer)))}
direct2idx = {v:k for k,v in enumerate(sorted(set(merge_df.director_str)))}

uidx2user = {k:v for k,v in enumerate(sorted(set(merge_df.user)))}
iidx2item = {k:v for k,v in enumerate(sorted(set(merge_df.item)))}
widx2wri = {k:v for k,v in enumerate(sorted(set(merge_df.writer)))}
didx2dir = {k:v for k,v in enumerate(sorted(set(merge_df.director_str)))}

merge_df.user = merge_df.user.map(user2idx)
merge_df.item = merge_df.item.map(item2idx)
merge_df.writer = merge_df.writer.map(writer2idx)
merge_df.director_str = merge_df.director_str.map(direct2idx)
merge_df.year = merge_df.year.map(int)

In [13]:
merge_df

,user,item,rating,year,writer,director_str,genre_embedding,dir_max_cnt,dir_sum_cnt,wri_max_cnt,wri_sum_cnt,seen_item_inter_mean,seen_item_inter_std
0,0,2505,1.0,2001,537,48,-0.010404,7743,51470,8936,21339,3719.351064,4316.497874
1,0,109,1.0,1995,0,1071,-0.010290,2092,4086,1107,1107,3719.351064,4316.497874
2,0,319,1.0,1993,270,280,-0.010649,966,1291,7743,19336,3719.351064,4316.497874
3,0,368,1.0,1970,381,983,-0.010579,2052,10798,2052,11529,3719.351064,4316.497874
4,0,1183,1.0,1982,172,196,-0.010351,2175,9200,2175,4005,3719.351064,4316.497874
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10285234,31359,1775,0.0,2000,146,460,-0.010054,159,455,747,906,3701.936306,4167.737972
10285235,31359,2213,0.0,2000,78,81,-0.010281,506,2209,4748,15704,3701.936306,4167.737972
10285236,31359,3007,0.0,2002,1800,200,-0.010508,338,864,218,313,3701.936306,4167.737972
10285237,31359,5201,0.0,2007,0,277,-0.010337,3571,10867,163,163,3701.936306,4167.737972


# Dataset

먼저 numerical feature와 categorical feature를 구분하는 과정을 거친다

In [14]:
# categorical feature 목록
cat_feats = ['user', 'item', 'year', 'writer', 'director_str']

def _convert_cat_features_dtype(df):
    for cat_feat in cat_feats:
        # df[cat_feat] = df[cat_feat].apply(str)
        df[cat_feat] = df[cat_feat].astype("category")
        # df[cat_feat] = df[cat_feat].astype(object)
    
def _convert_nu_features_dtype(df):
    cols = list(df.columns)
    for nu_feat in cols:
        if nu_feat in cat_feats:
            continue
        df[nu_feat] = df[nu_feat].astype(float)

# 함수에 사용할 df를 넣어주면 된다.
_convert_cat_features_dtype(merge_df)
_convert_nu_features_dtype(merge_df)

학습을 위해 train/valid split 한다

In [15]:
df_x_data = merge_df.drop(['rating'], axis=1)
df_y_data = merge_df['rating']

df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(df_x_data, df_y_data, test_size=0.1, random_state=43) #1

# Model

In [38]:
model = catboost.CatBoostClassifier(
    iterations= 2000, 
    random_seed= 43,
    custom_metric=["Precision","F1","Accuracy", 'Recall'],
    eval_metric="Accuracy",
    early_stopping_rounds=100,
    learning_rate=0.01,
    # cat_features=cat_feats,
    # loss_function="Logloss", # Binary Classification에서 적절하다고 함
    task_type="GPU",
    devices="0"
)

In [39]:
model.fit(df_x_train, df_y_train,
    cat_features=cat_feats,
    eval_set=(df_x_test, df_y_test),
    verbose=200 # n번의 iter 마다 output을 print
)

0:	learn: 0.7992184	test: 0.7997626	best: 0.7997626 (0)	total: 272ms	remaining: 9m 4s
200:	learn: 0.8068805	test: 0.8073978	best: 0.8073978 (200)	total: 1m 3s	remaining: 9m 31s
400:	learn: 0.8096924	test: 0.8114531	best: 0.8114531 (400)	total: 2m 20s	remaining: 9m 20s
600:	learn: 0.8140361	test: 0.8168434	best: 0.8168434 (600)	total: 3m 40s	remaining: 8m 32s
800:	learn: 0.8207122	test: 0.8262218	best: 0.8262287 (799)	total: 4m 58s	remaining: 7m 26s
1000:	learn: 0.8253841	test: 0.8319611	best: 0.8319611 (1000)	total: 6m 7s	remaining: 6m 6s
1200:	learn: 0.8285561	test: 0.8350529	best: 0.8350529 (1200)	total: 7m 9s	remaining: 4m 45s
1400:	learn: 0.8309642	test: 0.8370685	best: 0.8370685 (1400)	total: 8m 6s	remaining: 3m 27s
1600:	learn: 0.8325379	test: 0.8380952	best: 0.8380952 (1600)	total: 9m 2s	remaining: 2m 15s
1800:	learn: 0.8335802	test: 0.8387495	best: 0.8387553 (1796)	total: 9m 59s	remaining: 1m 6s
1999:	learn: 0.8342520	test: 0.8392133	best: 0.8392172 (1998)	total: 11m	remaining:

In [40]:
# model.predict_proba(df_x_test)[:,1]
a = model.predict(df_x_test)
b = df_y_test.values

cnt = 0
for i in range(221):
    if a[i] == b[i]:
        cnt+=1
print(cnt/221)

0.8280542986425339


분류 정확성이 80%정도 나온다

In [41]:
# Save model
model.save_model('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/model/saved/catboost_2000_model.bin') # catboost는 bin을 지원
# model.load_model('/opt/ml/input/code/model/saved/catboost_model.bin') # catboost는 bin을 지원

필요시 모델을 저장할 수 있다.  
catboost model은 .bin을 사용해야하며  
.pth등으로 저장할 수는 있으나 정식으로 지원하지는 않는다

# RecBole 모델 결과에 Catboost 적용

In [43]:
sample_df = pd.concat([ease_df, ffm_df, neu_df, recvae_df, sli_df], ignore_index=True)
sample_df

,user,item
0,11,4370
1,11,4886
2,11,47
3,11,32587
4,11,40815
...,...,...
1567935,138493,5349
1567936,138493,4226
1567937,138493,33794
1567938,138493,3578


사용할 모델들의 추천 결과를 concat으로 합친다

In [50]:
ref_df = sample_df
# ref_df['rating'] = 0 # 에러 방지를 위한 조치, column의 의미는 없음

In [51]:
ref_df = pd.merge(ref_df, full_year, on='item', how='left')
ref_df = pd.merge(ref_df, writ, on='item', how='left')
ref_df = pd.merge(ref_df, director, on='item', how='left')
ref_df = pd.merge(ref_df, genre, on='item', how='left')
ref_df = pd.merge(ref_df, pop_dir, on='item', how='left')
ref_df = pd.merge(ref_df, pop_wri, on='item', how='left')
ref_df = pd.merge(ref_df, user_seen, on='user', how='left')
ref_df.drop(['Unnamed: 0', 'writer_inter_x',	'writer_inter_y','writer_info'], axis=1, inplace=True)

ref_df.user = ref_df.user.map(user2idx)
ref_df.item = ref_df.item.map(item2idx)
ref_df.writer = ref_df.writer.map(writer2idx)
ref_df.director_str = ref_df.director_str.map(direct2idx)
ref_df.year = ref_df.year.map(int)

ref_df

,user,item,year,writer,director_str,genre_embedding,dir_max_cnt,dir_sum_cnt,wri_max_cnt,wri_sum_cnt,seen_item_inter_mean,seen_item_inter_std
0,0,2381,2001,37,36,-0.010420,14224,130434,3734,11878,3719.351064,4316.497874
1,0,2619,2001,286,1122,-0.010383,12105,33840,12217,60006,3719.351064,4316.497874
2,0,41,1995,225,61,-0.010262,18437,53385,13857,18569,3719.351064,4316.497874
3,0,4581,2005,205,39,-0.010371,18202,81880,10774,21692,3719.351064,4316.497874
4,0,4790,2005,1260,224,-0.010360,6494,15027,7050,37743,3719.351064,4316.497874
...,...,...,...,...,...,...,...,...,...,...,...,...
1567935,31359,2812,2002,783,91,-0.010371,9270,28705,11604,42542,3701.936306,4167.737972
1567936,31359,2304,2000,1498,917,-0.010262,14726,65526,14726,64298,3701.936306,4167.737972
1567937,31359,4646,2005,289,917,-0.010260,14726,65526,14726,64298,3701.936306,4167.737972
1567938,31359,1977,2000,919,95,-0.010375,13114,53160,13114,30070,3701.936306,4167.737972


In [52]:
cat_predict_rslt = model.predict(ref_df)
cat_predict_rslt_proba = model.predict_proba(ref_df) # [0일 확률, 1일 확률]
cat_predict_rslt_proba_1 = np.array([i[1] for i in cat_predict_rslt_proba])
cat_predict_rslt_proba_0 = np.array([i[0] for i in cat_predict_rslt_proba])

In [53]:
ref_df['cat_predict_rslt'] = cat_predict_rslt
ref_df['cat_predict_rslt_proba_1'] = cat_predict_rslt_proba_1
ref_df['cat_predict_rslt_proba_0'] = cat_predict_rslt_proba_0

In [54]:
ref_df = ref_df.sort_values('user')
ref_df

,user,item,year,writer,director_str,genre_embedding,dir_max_cnt,dir_sum_cnt,wri_max_cnt,wri_sum_cnt,seen_item_inter_mean,seen_item_inter_std,cat_predict_rslt,cat_predict_rslt_proba_1,cat_predict_rslt_proba_0
0,0,2381,2001,37,36,-0.010420,14224,130434,3734,11878,3719.351064,4316.497874,1.0,0.871737,0.128263
1254343,0,4790,2005,1260,224,-0.010360,6494,15027,7050,37743,3719.351064,4316.497874,1.0,0.939217,0.060783
1254344,0,2653,2001,572,247,-0.010271,11374,31362,11374,17841,3719.351064,4316.497874,1.0,0.966077,0.033923
1254345,0,4581,2005,205,39,-0.010371,18202,81880,10774,21692,3719.351064,4316.497874,1.0,0.976314,0.023686
1254346,0,43,1995,273,245,-0.010230,13789,33739,13789,16491,3719.351064,4316.497874,1.0,0.978958,0.021042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627136,31359,3563,2003,405,48,-0.010510,7743,51470,7146,19781,3701.936306,4167.737972,1.0,0.973833,0.026167
627137,31359,2345,2001,1357,811,-0.010666,3844,7723,3844,7723,3701.936306,4167.737972,1.0,0.877607,0.122393
627138,31359,2991,2002,1260,160,-0.010351,6431,25718,7050,37743,3701.936306,4167.737972,1.0,0.903805,0.096195
1567938,31359,1977,2000,919,95,-0.010375,13114,53160,13114,30070,3701.936306,4167.737972,1.0,0.975060,0.024940


- cat_predict_rslt : catboost가 1(positive)로 예측했는지, 0(negative)로 예측했는지
- cat_predict_rslt_proba_1 : cat_predict_rslt가 1일 확률 -> 높을 수록 좋다

RecBole 모델들이 예측한 결과를 Catboost가 다시 검수 하는 역할을 한다  
추천 결과의 우열을 가르거나, 올바른 추천이 아니라고 판단된 항목을 제거할 수도 있다

In [55]:
user_item_cat_df = ref_df[['user', 'item', 'cat_predict_rslt', 'cat_predict_rslt_proba_0', 'cat_predict_rslt_proba_1']]
user_item_cat_df

,user,item,cat_predict_rslt,cat_predict_rslt_proba_0,cat_predict_rslt_proba_1
0,0,2381,1.0,0.128263,0.871737
1254343,0,4790,1.0,0.060783,0.939217
1254344,0,2653,1.0,0.033923,0.966077
1254345,0,4581,1.0,0.023686,0.976314
1254346,0,43,1.0,0.021042,0.978958
...,...,...,...,...,...
627136,31359,3563,1.0,0.026167,0.973833
627137,31359,2345,1.0,0.122393,0.877607
627138,31359,2991,1.0,0.096195,0.903805
1567938,31359,1977,1.0,0.024940,0.975060


In [58]:
user_item_cat_df.user = user_item_cat_df.user.map(uidx2user)
user_item_cat_df.item = user_item_cat_df.item.map(iidx2item)
user_item_cat_df

/tmp/ipykernel_23881/2528849633.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_item_cat_df.user = user_item_cat_df.user.map(uidx2user)
/tmp/ipykernel_23881/2528849633.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_item_cat_df.item = user_item_cat_df.item.map(iidx2item)


,user,item,cat_predict_rslt,cat_predict_rslt_proba_0,cat_predict_rslt_proba_1
0,11,4370,1.0,0.128263,0.871737
1254343,11,40815,1.0,0.060783,0.939217
1254344,11,4963,1.0,0.033923,0.966077
1254345,11,32587,1.0,0.023686,0.976314
1254346,11,50,1.0,0.021042,0.978958
...,...,...,...,...,...
627136,138493,7147,1.0,0.026167,0.973833
627137,138493,4308,1.0,0.122393,0.877607
627138,138493,5816,1.0,0.096195,0.903805
1567938,138493,3578,1.0,0.024940,0.975060


In [64]:
# user_item_cat_df.loc[user_item_cat_df['user']==11]
cat_rslt_dict = {}
for i,j,k in list(zip(user_item_cat_df['user'].values, user_item_cat_df['item'].values, user_item_cat_df['cat_predict_rslt_proba_1'].values)):
    if i in cat_rslt_dict:
        if j in cat_rslt_dict[i]:
            continue
        else:
            cat_rslt_dict[i][j] = k
    else:
        cat_rslt_dict[i] = {}
        cat_rslt_dict[i][j] = k

In [66]:
cat_rslt_dict[11][50]

0.9789575233917895

## sub 만들기

모델이 예측한 결과들을 성능별로 가중치를 주고 Catboost_predict_proba 값을 곱해주는 것으로  
추천 대상에 대한 최종 스코어를 계산할 수 있다

In [67]:
rslt = {}
sample_rslt = pd.read_csv('/opt/ml/input/data/eval/sample_submission.csv')
for i in sample_rslt['user'].unique():
    rslt[i] = {}

In [73]:
# 모델의 추천 결과를 각각 가중치를 주어 스코어를 내는 부분

# ease : 0.1594
# recvae : 0.0963
# neu : 0.1048
# ffm : 0.0999
# sli : 0.1279

en_models = [ease_df, recvae_df, neu_df, ffm_df, sli_df]
en_w = [0.1594, 0.0963, 0.1048, 0.0999, 0.1279]

for k in range(len(en_models)):

    for i,j in list(zip(en_models[k]['user'].values, en_models[k]['item'].values)):
        if j in rslt[i]:
            rslt[i][j] += (en_w[k]*cat_rslt_dict[i][j])
        else:
            rslt[i][j] = (en_w[k]*cat_rslt_dict[i][j])

In [75]:
real_rslt = {}
for i in sample_rslt['user'].unique():
    real_rslt[i] = 0

for i in sample_rslt['user'].unique():
    real_rslt[i] = list(rslt[i].items())
    real_rslt[i].sort(key=lambda x:x[1], reverse=True)

ref = []
for k in sample_rslt['user'].unique():
    for i,j in real_rslt[k][:10]:
        ref.append([k,i])

en_sub = pd.DataFrame(ref,columns=['user', 'item'])
en_sub

In [80]:
en_sub.to_csv('/opt/ml/input/level2_movierecommendation_recsys-level2-recsys-08/en_EaseRecNeuFfmSli_cat.csv')